In [498]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd 
import bs4 as bs4
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import urllib


In [620]:
driver = webdriver.Chrome()
driver.get('https://hatcepc.honda.co.th/epc/index.jsf')
userInput = driver.find_element(By.ID,'loginForm:j_idt19')
userInput.send_keys('AD009252')
pwdInput = driver.find_element(By.ID,'loginForm:password1')
pwdInput.send_keys('Nutty@012')
pwdInput.send_keys(Keys.ENTER)

In [621]:
driver.find_element(By.XPATH,'/html/body/div[1]/div/span/form/div[1]/div[3]').click()
driver.find_element(By.XPATH,'/html/body/div[12]/div/ul/li[2]').click()
driver.find_element(By.XPATH,'/html/body/div[2]/form[1]/ul/li[4]/span/div[1]/div[2]/ul/li[3]').click()

In [526]:
PartList = ['01431SAET01','01432SAET01','01463S2A010','01463S87A01','01463SAA901','01463SHJA01',
            '01464T0AA02','01464T9AT01','01464TF0010','01465S9A000']

<font color='yellow' size=5>Change display items to 10</font>

In [555]:
def ChangeDisplayItem():
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)

    #To Select 10 rows per page.
    optionDisplay = soup.find_all('select')[3].find_all('option')
    optionDisplay = [x.text for x in optionDisplay]
    optionDisplay ='50' if '50' in optionDisplay else max(optionDisplay, key=int)
    optionDisplay
    dropdownMaxDisplay = driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/select')
    # Create a Select object
    select = Select(dropdownMaxDisplay)
    select.select_by_visible_text(optionDisplay)

In [625]:
def wait_for_ajax(driver):
    wait = WebDriverWait(driver, 10)
    try:
        wait.until(lambda driver: driver.execute_script('return jQuery.active') == 0)
        wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
    except Exception as e:
        pass  # Handle the exception or pass



In [643]:
def InputSearchBox(PartNo):        
        wait = WebDriverWait(driver, 10)  # Adjust the timeout as needed
        wait.until(EC.element_to_be_clickable((By.ID, 'searchValue')))
        time.sleep(5) 
        driver.find_element(By.ID,'searchValue').clear()
        driver.find_element(By.ID,'searchValue').send_keys(PartNo)
        driver.find_element(By.ID,'searchValue').send_keys(Keys.ENTER)
        print('input '+PartNo)

In [630]:
def getModel(soup):
    wait = WebDriverWait(driver, 30) 
    try:
        wait.until(EC.presence_of_element_located((By.ID, 'searchResultPanel')))
        MatchPartList = soup.find_all('tbody',{'id':'itemSearchTable_data'})[0].find_all('tr')
        MatchPartList = pd.DataFrame([x.find_all('td')[0].text for x in MatchPartList] ,columns=['Part'])
        MatchPartList['PartID'] = MatchPartList['Part'].apply(lambda x : x.split('(',1)[0].split(' - ')[0])
        MatchPartList['CarModel'] = MatchPartList['Part'].apply(lambda x : x.split('(',1)[1].split(')')[0] )
        MatchPartList['PartDescription'] = MatchPartList['Part'].apply(lambda x : x.split(')',1)[1])
        MatchPartList = MatchPartList.loc[MatchPartList['CarModel'].apply(lambda x : x.find(' TH '))>=0]
    except TimeoutException:
        print("Timeout occurred! Performing alternative steps.")
    finally:
        print('done'+str(len(MatchPartList)))
    return(MatchPartList)

In [560]:
def checkData(partCheck):
    # Set your timeout period (in seconds)
    timeout = 20
    start_time = time.time()
    partCheck = partCheck.strip()
    
    found = False
    while True:
        try:
            data = driver.page_source
            soup = bs4.BeautifulSoup(data)
            # Attempt your operation
            # First condition using BeautifulSoup
            text_beautifulsoup = soup.find_all('tbody', {'id': 'itemSearchTable_data'})[0].find_all('tr')[0].find_all('td')[0].text.replace('-', '')
            condition1 = partCheck in text_beautifulsoup

            # Second condition using Selenium
            text_selenium = driver.find_element(By.XPATH, '/html/body/div[2]/form[2]/div/fieldset/div/div/div[2]/table/tbody/tr').text
            condition2 = text_selenium == 'ไม่พบรายการที่ตรงกัน ...'
            result = condition1 or condition2
            #print('selenium = '+text_selenium)
            #print('soup='+text_beautifulsoup)
            if result:
                found = True
                
                if text_selenium == 'ไม่พบรายการที่ตรงกัน ...':
                    print('empty row '+partCheck)
                    time.sleep(1)
                    return(condition2, condition1)
                else:
                    print("Found the desired text. "+partCheck)
                    return(condition2, condition1)
                break
        except IndexError:
            print("Element not found in the structure. Retrying...")
        except AttributeError:
            print("A 'find' method was called on a NoneType object. Retrying...")

        # Check if the timeout period has been exceeded
        if time.time() - start_time > timeout:
            print("Timeout reached.")
            return(False)
            break

        # Optional: add a short delay before retrying
        print('retry '+partCheck )
        #time.sleep(1)

In [511]:
PartList

['01431SAET01',
 '01432SAET01',
 '01463S2A010',
 '01463S87A01',
 '01463SAA901',
 '01463SHJA01',
 '01464T0AA02',
 '01464T9AT01',
 '01464TF0010',
 '01465S9A000']

<font color='yellow' size=5>Simulate Running</font>

In [539]:
InputSearchBox(PartList[0])

input 01431SAET01


In [540]:
checkData(PartList[0])

Found the desired text. 01431SAET01


(False, True)

In [541]:
ChangeDisplayItem()

In [542]:
data = driver.page_source
soup = bs4.BeautifulSoup(data)

In [543]:
MatchPartList = soup.find_all('tbody',{'id':'itemSearchTable_data'})[0].find_all('tr')
MatchPartList = pd.DataFrame([x.find_all('td')[0].text for x in MatchPartList] ,columns=['Part'])
MatchPartList['PartID'] = MatchPartList['Part'].apply(lambda x : x.split('(',1)[0].split(' - ')[0])
MatchPartList['CarModel'] = MatchPartList['Part'].apply(lambda x : x.split('(',1)[1].split(')')[0] )
MatchPartList['PartDescription'] = MatchPartList['Part'].apply(lambda x : x.split(')',1)[1])
MatchPartList.loc[MatchPartList['CarModel'].apply(lambda x : x.find(' TH '))>=0]

In [584]:
data = driver.page_source
soup = bs4.BeautifulSoup(data)
nLoop = len(soup.find_all('span',{'class':'ui-paginator-pages'})[0].find_all('a'))
for page in range(1, nLoop + 1):
    print('Page '+str(page))
#driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/span[2]')


Page 1
Page 2


In [585]:
def findNLoop():
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    nLoop = len(soup.find_all('span',{'class':'ui-paginator-pages'})[0].find_all('a'))
    return(nLoop)

In [580]:
len(soup.find_all('span',{'class':'ui-paginator-pages'})[0].find_all('a'))

2

In [589]:
driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/a[3]/span').click()
#driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/a[3]/span').click()

<font color='lightgreen' size=5>Run Looping</font>

In [646]:
ResultData = pd.DataFrame()
FirstRun = True
for partRunning in PartList[:10]:
    InputSearchBox(partRunning)
    #time.sleep(5)

    FoundNoUse, FoundModel = checkData(partRunning)
    if FirstRun and FoundModel:        
        ChangeDisplayItem()
        FirstRun=False
        print('first run')

    #Check number of pages to be click through.
    nLoop = findNLoop()
    for page in range(1, nLoop + 1):
        print('Page '+str(page)+str(FoundNoUse)+str(FoundModel))
        if FoundModel:
            data = driver.page_source
            soup = bs4.BeautifulSoup(data)
            getModelList = getModel(soup)            
        elif FoundNoUse:
            getModelList = pd.DataFrame(columns=['Part', 'PartID', 'CarModel', 'PartDescription'])
            getModelList.loc[0] = [None, partRunning, None, None]
        else:
            print('Timeout.....')
        
        ResultData = pd.concat([ ResultData, getModelList] , ignore_index=True)

        #Click Next page
        if page < nLoop:
            wait = WebDriverWait(driver, 60)
            wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.ui-paginator-next.ui-state-default.ui-corner-all')))
            #wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'ui-icon ui-icon-seek-next')))
            element_to_click = driver.find_element(By.XPATH, '/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/a[3]/span')
            driver.execute_script("arguments[0].click();", element_to_click)

            #driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/a[3]/span').click()
            FoundNoUse, FoundModel = checkData(partRunning)        

    next 
    #wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.ui-button-text ui-c')))   
    #wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/form[2]/div/fieldset/div/div/div[1]/button'))).click()
    button = driver.find_element(By.XPATH, '/html/body/div[2]/form[2]/div/fieldset/div/div/div[1]/button')
    driver.execute_script("arguments[0].click();", button)
    
    #driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[1]/button').click()
    
    

input 01431SAET01
Found the desired text. 01431SAET01
first run
Page 1FalseTrue
done32
Found the desired text. 01431SAET01
Page 2FalseTrue
done32
input 01432SAET01
retry 01432SAET01
retry 01432SAET01
retry 01432SAET01
retry 01432SAET01
retry 01432SAET01
retry 01432SAET01
Found the desired text. 01432SAET01
Page 1FalseTrue
done32
Found the desired text. 01432SAET01
Page 2FalseTrue
done32
input 01463S2A010
retry 01463S2A010
retry 01463S2A010
retry 01463S2A010
retry 01463S2A010
retry 01463S2A010
retry 01463S2A010
retry 01463S2A010
retry 01463S2A010
retry 01463S2A010
retry 01463S2A010
retry 01463S2A010
retry 01463S2A010
retry 01463S2A010
retry 01463S2A010
Found the desired text. 01463S2A010
Page 1FalseTrue
done12
Found the desired text. 01463S2A010
Page 2FalseTrue
done12
Found the desired text. 01463S2A010
Page 3FalseTrue
done12
Found the desired text. 01463S2A010
Page 4FalseTrue
done12
Found the desired text. 01463S2A010
Page 5FalseTrue
done12
Found the desired text. 01463S2A010
Page 6Fal

TypeError: cannot unpack non-iterable bool object

In [634]:
ResultData

,Part,PartID,CarModel,PartDescription
0,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2007 TH - ...,01431-SAE-T01,JAZZ 5Dr 2007 TH - [ 5MT - S ],B-19 - REAR BRAKE(DRUM)
1,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2007 TH - ...,01431-SAE-T01,JAZZ 5Dr 2007 TH - [ 5MT - S ],B-19 - REAR BRAKE(DRUM)
2,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2007 TH - ...,01431-SAE-T01,JAZZ 5Dr 2007 TH - [ CVT - S ],B-19 - REAR BRAKE(DRUM)
3,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2007 TH - ...,01431-SAE-T01,JAZZ 5Dr 2007 TH - [ CVT - S ],B-19 - REAR BRAKE(DRUM)
4,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2007 TH - ...,01431-SAE-T01,JAZZ 5Dr 2007 TH - [ 5MT - S BEIGE ],B-19 - REAR BRAKE(DRUM)
5,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2007 TH - ...,01431-SAE-T01,JAZZ 5Dr 2007 TH - [ 5MT - S BEIGE ],B-19 - REAR BRAKE(DRUM)
6,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2007 TH - ...,01431-SAE-T01,JAZZ 5Dr 2007 TH - [ CVT - S BEIGE ],B-19 - REAR BRAKE(DRUM)
7,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2007 TH - ...,01431-SAE-T01,JAZZ 5Dr 2007 TH - [ CVT - S BEIGE ],B-19 - REAR BRAKE(DRUM)
8,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2006 TH - ...,01431-SAE-T01,JAZZ 5Dr 2006 TH - [ 5MT - S ],B-19 - REAR BRAKE(DRUM)
9,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2006 TH - ...,01431-SAE-T01,JAZZ 5Dr 2006 TH - [ 5MT - S ],B-19 - REAR BRAKE(DRUM)


In [563]:
ResultData[['PartID','CarModel','PartDescription']].drop_duplicates(subset=['PartID'])

,PartID,CarModel,PartDescription
0,01431-SAE-T01,JAZZ 5Dr 2007 TH - [ 5MT - S ],B-19 - REAR BRAKE(DRUM)
50,01432-SAE-T01,JAZZ 5Dr 2007 TH - [ 5MT - S ],B-19 - REAR BRAKE(DRUM)
100,01463-S2A-010,CITY 4Dr 2019 FO - [ CVT - VTI-S ],B-22 - FRONT BRAKE
150,01463S87A01,None,None
151,01463-SAA-901,BRIO 5Dr 2021 PH - [ 5MT - S ],B-22 - FRONT BRAKE
201,01463SHJA01,None,None
202,01464-T0A-A02,CR-V 5Dr 2014 KU - [ 5AT - 24RVSI ],B-25-21 - BRAKE LINES(2.0L)(2.4L)(V SA)(RH)
252,01464-T9A-T01,CITY 4Dr 2019 FO - [ CVT - VTI-S ],B-25-21 - BRAKE LINES(VSA)(LH)
302,01464-TF0-010,JAZZ 5Dr 2014 KQ - [ 5MT - 1.3LX ],B-25-20 - BRAKE PIPE/HOSE(VSA)
352,01465-S9A-000,CR-V 5Dr 2002 TH - [ 4AT - RVI ],B-25-10 - BRAKE LINES (ABS)


In [363]:
# Set your timeout period (in seconds)
timeout = 10
start_time = time.time()

found = False
while True:
    try:
        # Attempt your operation
        result = soup.find_all('tbody', {'id': 'itemSearchTable_data'})[0].find_all('tr')[0].find_all('td')[0].text.replace('-', '').find('01465S9A000') >= 0
        if result:
            found = True
            print("Found the desired text.")
            break
    except IndexError:
        print("Element not found in the structure. Retrying...")
    except AttributeError:
        print("A 'find' method was called on a NoneType object. Retrying...")

    # Check if the timeout period has been exceeded
    if time.time() - start_time > timeout:
        print("Timeout reached.")
        break

    # Optional: add a short delay before retrying
    time.sleep(1)

Found the desired text.


In [366]:
partRunning

'01465S9A000'

In [372]:
checkData('01465S9A000')

Found the desired text.


True

In [217]:
driver.find_element(By.ID,'searchValue').send_keys(PartList[0])
driver.find_element(By.ID,'searchValue').send_keys(Keys.ENTER)

In [ ]:
ChangeDisplayItem()

In [242]:
data = driver.page_source
soup = bs4.BeautifulSoup(data)

In [258]:
MatchPartList = soup.find_all('tbody',{'id':'itemSearchTable_data'})[0].find_all('tr')
MatchPartList = pd.DataFrame(([x.find_all('td')[0].text for x in MatchPartList]) ,columns=['Part'])
MatchPartList['PartID'] = MatchPartList['Part'].apply(lambda x : x.split('(',1)[0].split(' - ')[0])
MatchPartList['CarModel'] = MatchPartList['Part'].apply(lambda x : x.split('(',1)[1].split(')')[0] )
MatchPartList['PartDescription'] = MatchPartList['Part'].apply(lambda x : x.split(')',1)[1])

In [249]:
MatchPartList = soup.find_all('tbody',{'id':'itemSearchTable_data'})[0].find_all('tr')
MatchPartList = pd.DataFrame(list(set([x.find_all('td')[0].text for x in MatchPartList])) ,columns=['Part'])

In [256]:
MatchPartList = soup.find_all('tbody',{'id':'itemSearchTable_data'})[0].find_all('tr')

In [255]:
soup.find_all('tbody',{'id':'itemSearchTable_data'})[0].find_all('tr')

[<tr class="ui-widget-content ui-datatable-even" data-ri="0" role="row"><td class="Wid40" role="gridcell"><span class="ui-column-title"></span><span class="FontBold">01431-SAE-T01</span> - 18SAA4T1 ( JAZZ 5Dr 2007 TH - [ 5MT - S ] )<br/>B-19 - REAR BRAKE(DRUM)</td><td class="Wid30" role="gridcell"><span class="ui-column-title"></span>SHOE KIT, R. RR. BRAKE</td><td class="Wid20" role="gridcell"><span class="ui-column-title"></span><input aria-disabled="false" aria-readonly="false" class="ui-inputfield ui-inputtext ui-widget ui-state-default ui-corner-all Wid10 ui-state-filled" id="itemSearchTable:0:itemList" name="itemSearchTable:0:itemList" onkeyup="PrimeFaces.bcn(this,event,[function(event){return updateQtyToCart(event, this, '1')},function(event){PrimeFaces.ab({s:&quot;itemSearchTable:0:itemList&quot;,e:&quot;keyup&quot;,f:&quot;itemSearchResultForm&quot;,p:&quot;itemSearchTable:0:itemList&quot;,u:&quot;itemSearchTable topMenuForm mainPanel dialogPartAssociation menuForm overlayCartL

In [257]:
pd.DataFrame(list(set([x.find_all('td')[0].text for x in MatchPartList])) ,columns=['Part'])

,Part
0,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2007 TH - ...
1,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2007 TH - ...
2,01431-SAE-T01 - 18SAA4T1 ( JAZZ 5Dr 2007 TH - ...
